In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [51]:
import os

import netcdf_scm.io
import netcdf_scm.stitching
import tqdm.autonotebook as tqdman
import xarray as xr
from netcdf_scm.iris_cube_wrappers import ScmCube

<IPython.core.display.Javascript object>

In [77]:
all_errors = [
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r3i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r3i1p1f1_gr_202001-202412.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r10i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r10i1p1f1_gr_202001-202412.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r4i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r4i1p1f1_gr_202001-202412.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r2i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r2i1p1f1_gr_202001-202412.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp585/r1i1p1f1/Amon/tas/gn/v20200915/netcdf-scm_tas_Amon_IITM-ESM_ssp585_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp585/r1i1p1f1/Amon/tas/gr/v20201117/netcdf-scm_tas_Amon_E3SM-1-1_ssp585_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp126/r1i1p1f1/Amon/tas/gn/v20201112/netcdf-scm_tas_Amon_IITM-ESM_ssp126_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/netcdf-scm_tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp245/r1i1p1f1/Amon/tas/gn/v20190731/netcdf-scm_tas_Amon_NESM3_ssp245_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/ssp370/r2i1p1f1/Amon/tas/gn/v20190628/netcdf-scm_tas_Amon_MPI-ESM-1-2-HAM_ssp370_r2i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r1i1p1f1/Amon/tas/gr/v20201109/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r1i1p1f1_gr_202001-202412.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp585/r1i1p1f1/Amon/tas/gn/v20190728/netcdf-scm_tas_Amon_NESM3_ssp585_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-CC/ssp245/r1i1p1f1/Amon/tas/gr/v20210113/netcdf-scm_tas_Amon_EC-Earth3-CC_ssp245_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-CC/ssp585/r1i1p1f1/Amon/tas/gr/v20210113/netcdf-scm_tas_Amon_EC-Earth3-CC_ssp585_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp245/r2i1p1f1/Amon/tas/gn/v20190731/netcdf-scm_tas_Amon_NESM3_ssp245_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Amon/tas/gn/v20190314/netcdf-scm_tas_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp370/r1i1p1f1/Amon/tas/gn/v20200915/netcdf-scm_tas_Amon_IITM-ESM_ssp370_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/KIOST/KIOST-ESM/ssp585/r1i1p1f1/Amon/tas/gr1/v20191106/netcdf-scm_tas_Amon_KIOST-ESM_ssp585_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/Amon/tas/gn/v20190314/netcdf-scm_tas_Amon_BCC-CSM2-MR_ssp370_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp126/r1i1p1f1/Amon/tas/gn/v20190731/netcdf-scm_tas_Amon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp585/r2i1p1f1/Amon/tas/gn/v20190731/netcdf-scm_tas_Amon_NESM3_ssp585_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp370/r2i1p1f1/Amon/tas/gn/v20190529/netcdf-scm_tas_Amon_AWI-CM-1-1-MR_ssp370_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/INM/INM-CM5-0/ssp370/r2i1p1f1/Amon/tas/gr1/v20190712/netcdf-scm_tas_Amon_INM-CM5-0_ssp370_r2i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp126/r1i1p1f1/Amon/tas/gn/v20190314/netcdf-scm_tas_Amon_BCC-CSM2-MR_ssp126_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/INM/INM-CM5-0/ssp370/r4i1p1f1/Amon/tas/gr1/v20190723/netcdf-scm_tas_Amon_INM-CM5-0_ssp370_r4i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/KIOST/KIOST-ESM/ssp245/r1i1p1f1/Amon/tas/gr1/v20191106/netcdf-scm_tas_Amon_KIOST-ESM_ssp245_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp245/r1i1p1f1/Amon/tas/gn/v20190314/netcdf-scm_tas_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp370/r4i1p1f1/Amon/tas/gn/v20190529/netcdf-scm_tas_Amon_AWI-CM-1-1-MR_ssp370_r4i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/INM/INM-CM5-0/ssp370/r3i1p1f1/Amon/tas/gr1/v20190715/netcdf-scm_tas_Amon_INM-CM5-0_ssp370_r3i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DWD/MPI-ESM1-2-HR/ssp585/r2i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp585_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/KIOST/KIOST-ESM/ssp126/r1i1p1f1/Amon/tas/gr1/v20191106/netcdf-scm_tas_Amon_KIOST-ESM_ssp126_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NUIST/NESM3/ssp126/r2i1p1f1/Amon/tas/gn/v20190801/netcdf-scm_tas_Amon_NESM3_ssp126_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r4i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp370_r4i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r10i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp370_r10i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DWD/MPI-ESM1-2-HR/ssp126/r2i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp126_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r2i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp370_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp370/r3i1p1f1/Amon/tas/gn/v20190529/netcdf-scm_tas_Amon_AWI-CM-1-1-MR_ssp370_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r3i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp370_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/THU/CIESM/ssp126/r1i1p1f1/Amon/tas/gr/v20200417/netcdf-scm_tas_Amon_CIESM_ssp126_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp245/r2i1p1f1/Amon/tas/gn/v20190710/netcdf-scm_tas_Amon_MPI-ESM1-2-HR_ssp245_r2i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/THU/CIESM/ssp585/r1i1p1f1/Amon/tas/gr/v20200605/netcdf-scm_tas_Amon_CIESM_ssp585_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/THU/CIESM/ssp245/r1i1p1f1/Amon/tas/gr/v20200417/netcdf-scm_tas_Amon_CIESM_ssp245_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp585/r1i1p1f1/Amon/tas/gn/v20200702/netcdf-scm_tas_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-229912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp370_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-CM4_ssp585_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp245/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-CM4_ssp245_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp119/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp119_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp126/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp126_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp585_r1i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp245/r1i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp245_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp245/r3i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp245_r3i1p1f1_gr1_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp245/r1i1p1f1/Amon/tas/gn/v20190708/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Amon/tas/gn/v20190708/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp119/r1i1p1f1/Amon/tas/gn/v20190708/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp119_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp126/r1i1p1f1/Amon/tas/gn/v20190708/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp126_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp126/r2i1p1f1/Amon/tas/gn/v20190924/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp126_r2i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r1i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp370_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp245/r3i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp245_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp370/r2i1p1f1/Amon/tas/gn/v20190924/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp370_r2i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r3i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp585_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/NOAA-GFDL/GFDL-ESM4/ssp370-lowNTCF/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-ESM4_ssp370-lowNTCF_r1i1p1f1_gr1_201501-205912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r2i1p1f1/Amon/tas/gn/v20190924/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp585_r2i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp534-over/r2i1p1f1/Amon/tas/gn/v20200211/netcdf-scm_tas_Amon_CESM2-WACCM_ssp534-over_r2i1p1f1_gn_204001-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r3i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp370_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp126/r3i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp126_r3i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp119/r2i1p1f1/Amon/tas/gn/v20190924/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp119_r2i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp370/r1i1p1f1/Amon/tas/gn/v20190708/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp370_r1i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r1i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp585_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20200701/netcdf-scm_tas_Amon_CESM2-WACCM_ssp534-over_r1i1p1f1_gn_204001-229912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp245/r1i1p1f1/Amon/tas/gr/v20191013/netcdf-scm_tas_Amon_FGOALS-f3-L_ssp245_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp245/r1i1p1f1/Amon/tas/gn/v20201124/netcdf-scm_tas_Amon_TaiESM1_ssp245_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370-lowNTCF/r3i1p1f1/Amon/tas/gn/v20190624/netcdf-scm_tas_Amon_BCC-ESM1_ssp370-lowNTCF_r3i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp126/r1i1p1f1/Amon/tas/gn/v20201228/netcdf-scm_tas_Amon_CAS-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp126/r1i1p1f1/Amon/tas/gn/v20201124/netcdf-scm_tas_Amon_TaiESM1_ssp126_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp245/r2i1p1f1/Amon/tas/gn/v20190924/netcdf-scm_tas_Amon_CAMS-CSM1-0_ssp245_r2i1p1f1_gn_201501-209912.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp370/r1i1p1f1/Amon/tas/gr/v20191013/netcdf-scm_tas_Amon_FGOALS-f3-L_ssp370_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/ssp370-lowNTCF/r2i1p1f1/Amon/tas/gn/v20190628/netcdf-scm_tas_Amon_MPI-ESM-1-2-HAM_ssp370-lowNTCF_r2i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r3i1p1f1/Amon/tas/gn/v20190702/netcdf-scm_tas_Amon_BCC-ESM1_ssp370_r3i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1p1f1/Amon/tas/gn/v20190624/netcdf-scm_tas_Amon_BCC-ESM1_ssp370_r1i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp370/r1i1p1f1/Amon/tas/gn/v20201014/netcdf-scm_tas_Amon_TaiESM1_ssp370_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp126/r1i1p1f1/Amon/tas/gr/v20191013/netcdf-scm_tas_Amon_FGOALS-f3-L_ssp126_r1i1p1f1_gr_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370-lowNTCF/r1i1p1f1/Amon/tas/gn/v20190624/netcdf-scm_tas_Amon_BCC-ESM1_ssp370-lowNTCF_r1i1p1f1_gn_201501-205512.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp585/r1i1p1f1/Amon/tas/gn/v20200901/netcdf-scm_tas_Amon_TaiESM1_ssp585_r1i1p1f1_gn_201501-210012.nc",
    "./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp585/r1i1p1f1/Amon/tas/gr/v20191013/netcdf-scm_tas_Amon_FGOALS-f3-L_ssp585_r1i1p1f1_gr_201501-210012.nc",
]

<IPython.core.display.Javascript object>

In [78]:
def _load_helper_and_scm_cubes(path):
    scm_cubes = {}

    data = xr.open_dataset(path)
    data.load()  # get everything in memory

    # Must be kept until https://github.com/pandas-dev/pandas/issues/37071
    # is solved
    if data["time"].encoding["units"] == "days since 1-01-01 00:00:00":
        data["time"].encoding["units"] = "days since 0001-01-01 00:00:00"

    for _, darray in data.data_vars.items():
        try:
            region = darray.attrs["region"]
        except KeyError:
            # bnds or some other unclassified variable
            continue
        scm_cubes[region] = ScmCube()

        scm_cubes[region].cube = darray.to_iris()
        scm_cubes[region].cube.attributes = {
            **scm_cubes[region].cube.attributes,
            **data.attrs,
        }
        break

    # take any cube as base for now, not sure how to really handle this so will
    # leave like this for now and only make this method public when I work it
    # out...
    loaded = list(scm_cubes.values())[0]

    return loaded, scm_cubes


<IPython.core.display.Javascript object>

In [79]:
netcdf_scm.io._load_helper_and_scm_cubes = _load_helper_and_scm_cubes

<IPython.core.display.Javascript object>

In [98]:
contacts = []
for f in tqdman.tqdm(all_errors):
    if "GFDL-CM4" not in f:
        continue

    source = netcdf_scm.io.load_scmrun(f)
    parent_replacements = netcdf_scm.stitching.get_parent_replacements(source)
    try:
        parent_file = netcdf_scm.stitching.get_parent_file_path(
            f, parent_replacements, "CMIP6Output"
        )
    except OSError as exc:
        print(exc)
        print()
        print("---")
        continue

    parent = netcdf_scm.io.load_scmrun(parent_file)

    picontrol_replacements = netcdf_scm.stitching.get_parent_replacements(parent)
    try:
        picontrol_file = netcdf_scm.stitching.get_parent_file_path(
            parent_file, picontrol_replacements, "CMIP6Output"
        )
    except OSError as exc:
        #         print(exc)
        #         print()
        #         print("---")
        continue

    try:
        contact = xr.load_dataset(parent_file).attrs["contact"]
    except KeyError:
        #         print("No contact for {}".format(parent_file))
        #         print()
        #         print("---")
        continue

    if contact in contacts:
        continue
    else:
        contacts.append(contact)

    print("dataset id: {}".format(".".join(parent_file.split(os.sep)[3:-1])))
    !ncdump -h {parent_file} | grep :contact
    !ncdump -h {parent_file} | grep branch_time_in_parent
    !ncdump -h {parent_file} | grep parent_time_units
    print(
        "		:branch time in human readable form = {}".format(
            netcdf_scm.stitching.get_branch_time(parent)
        )
    )

    print()
    print(
        "piControl dataset id: {}".format(".".join(picontrol_file.split(os.sep)[3:-1]))
    )
    picontrol_start = os.path.basename(picontrol_file).split("_")[-1].split("-")[0]
    picontrol_end = (
        os.path.basename(picontrol_file)
        .split("_")[-1]
        .split("-")[-1]
        .replace(".nc", "")
    )
    print(
        "		:piControl start time: {}-{}".format(
            picontrol_start[:4], picontrol_start[4:]
        )
    )
    print("		:piControl end time: {}-{}".format(picontrol_end[:4], picontrol_end[4:]))
    print("---")
    print()
#     break

  0%|          | 0/84 [00:00<?, ?it/s]

No parent data (historical) available for ./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-CM4_ssp585_r1i1p1f1_gr1_201501-210012.nc, we looked in ./country-crunch/netcdf-scm-crunched/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/Amon/tas/gr1/*/netcdf-scm_tas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_*.nc and ./country-crunch/netcdf-scm-crunched/CMIP6/CMIP/*/GFDL-CM4/historical/r1i1p1f1/Amon/tas/gr1/*/netcdf-scm_tas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_*.nc

---
No parent data (historical) available for ./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp245/r1i1p1f1/Amon/tas/gr1/v20180701/netcdf-scm_tas_Amon_GFDL-CM4_ssp245_r1i1p1f1_gr1_201501-210012.nc, we looked in ./country-crunch/netcdf-scm-crunched/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/Amon/tas/gr1/*/netcdf-scm_tas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_*.nc and ./country-crunch/netcdf-scm-crunched/CMIP6/CM

<IPython.core.display.Javascript object>